# 🎬 AI Video Search Pipeline - Google Colab

Complete video search system with CLIP embeddings, auto-labeling, and semantic search.

**Features:**
- ✅ CLIP-based video embeddings (GPU accelerated)
- ✅ Auto-labeling (actions, objects, captions)
- ✅ FAISS vector search
- ✅ FastAPI server with web UI
- ✅ Process 13K+ videos (UCF101 dataset)

**Runtime:** GPU (T4 recommended)

## 📦 Step 1: Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/YOUR_USERNAME/Rag_Video_search_Pipeline.git
%cd Rag_Video_search_Pipeline

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
!pip install -q pyngrok  # For exposing API to internet

## 🔑 Step 2: Set Hugging Face Token

In [ ]:
import os
from google.colab import userdata

# Option 1: Use Colab Secrets (recommended)
# Go to: 🔑 icon on left sidebar → Add secret: HF_TOKEN
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ Using HF_TOKEN from Colab secrets")
except:
    # Option 2: Set directly (less secure)
    HF_TOKEN = ""  # Replace with your token
    print("⚠️  Using hardcoded token")

os.environ["HF_TOKEN"] = HF_TOKEN
print(f"Token set: {HF_TOKEN[:10]}...")

## 📥 Step 3: Download Dataset

Choose one:
- **Option A:** Small subset (200 videos, ~5 min)
- **Option B:** Full UCF101 (13K videos, ~2 hours)

In [ ]:
# Option A: Download UCF101 subset (200 videos)
!mkdir -p ground_clips_mp4
!wget -q https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!unrar x -y UCF101.rar

# Create subset
!python process_subset.py

In [ ]:
# Option B: Download full UCF101 (13,320 videos - 6.5GB)
# Uncomment to use:

# !mkdir -p ground_clips_mp4
# !wget -q https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
# !unrar x -y UCF101.rar ground_clips_mp4/

# # Convert AVI to MP4
# !python convert_ucf101_videos.py

In [ ]:
# Check dataset
!python check_dataset.py

## 🚀 Step 4: Download Models (One-time)

In [ ]:
# Download CLIP model
!python download_clip_model.py

In [ ]:
# Download BLIP captioning model
!python download_blip_model.py

## 🎬 Step 5: Process Videos

This will:
1. Extract keyframes
2. Generate CLIP embeddings (GPU accelerated)
3. Auto-label videos (actions, objects, captions)
4. Build FAISS search index

**Time:**
- 200 videos: ~10-15 minutes
- 13K videos: ~2-3 hours

In [ ]:
# Process videos with auto-labeling
!python run_pipeline.py --enable-labeling

## 🔍 Step 6: Test Search (Local)

In [ ]:
# Quick search test
!python verify_search_works.py

## 🌐 Step 7: Start API Server (Public URL)

In [ ]:
# Install ngrok for public URL
!pip install -q pyngrok

from pyngrok import ngrok
import threading
import subprocess
import time

# Set ngrok auth token (get free token from https://ngrok.com)
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")  # Uncomment and add your token

# Start API server in background
def start_api():
    subprocess.run(["python", "run_api.py"])

api_thread = threading.Thread(target=start_api, daemon=True)
api_thread.start()

# Wait for server to start
time.sleep(10)

# Create public URL
public_url = ngrok.connect(8081)
print("\n" + "="*70)
print("🌐 Public URL for your Video Search API:")
print(f"   {public_url}")
print("\n📱 Web Interface:")
print(f"   {public_url}/static/index.html")
print("\n📖 API Docs:")
print(f"   {public_url}/docs")
print("="*70)
print("\n⚠️  Keep this cell running to maintain the connection!")
print("   Stop the cell to shut down the server.")

## 🧪 Step 8: Test API

In [ ]:
import requests
import json

# Test search
url = "http://localhost:8081/v1/search"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": "changeme"
}
data = {
    "query": "person playing basketball",
    "top_k": 5
}

response = requests.post(url, headers=headers, json=data)
results = response.json()

print("🔍 Search Results:")
print("="*70)
for i, result in enumerate(results.get("results", []), 1):
    print(f"\n{i}. {result['label']}")
    print(f"   Score: {result['score']*100:.1f}%")
    print(f"   Time: {result['start_time']:.1f}s - {result['end_time']:.1f}s")
    if 'auto_labels' in result:
        print(f"   Caption: {result['auto_labels'].get('caption', 'N/A')}")
        print(f"   Objects: {', '.join(result['auto_labels'].get('objects', [])[:5])}")

## 💾 Step 9: Save Results to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy processed data to Drive
!mkdir -p /content/drive/MyDrive/VideoSearch
!cp -r data /content/drive/MyDrive/VideoSearch/
!cp config/pipeline.yaml /content/drive/MyDrive/VideoSearch/

print("✅ Saved to Google Drive: MyDrive/VideoSearch/")
print("   - data/ (embeddings, index, metadata)")
print("   - pipeline.yaml (config)")

## 📊 Step 10: Performance Stats

In [ ]:
import json
from pathlib import Path

# Load metadata
metadata_path = Path("data/processed/metadata.json")
if metadata_path.exists():
    with open(metadata_path) as f:
        metadata = json.load(f)
    
    print("📊 Pipeline Statistics")
    print("="*70)
    print(f"Total videos processed: {len(metadata)}")
    
    # Count labels
    labels = {}
    for entry in metadata:
        label = entry.get('label', 'unknown')
        labels[label] = labels.get(label, 0) + 1
    
    print(f"Unique action categories: {len(labels)}")
    print(f"\nTop 10 categories:")
    for label, count in sorted(labels.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {label:20s} - {count} videos")
    
    # Check embeddings
    embeddings_dir = Path("data/processed/embeddings")
    if embeddings_dir.exists():
        emb_count = len(list(embeddings_dir.glob("*.npy")))
        print(f"\nEmbeddings generated: {emb_count}")
    
    # Check FAISS index
    index_path = Path("data/index/faiss.index")
    if index_path.exists():
        size_mb = index_path.stat().st_size / 1024 / 1024
        print(f"FAISS index size: {size_mb:.1f} MB")
    
    print("\n✅ Pipeline complete and ready for search!")
else:
    print("❌ No metadata found. Run pipeline first.")

## 🎯 Quick Commands Reference

```bash
# Check dataset
!python check_dataset.py

# Process videos
!python run_pipeline.py --enable-labeling

# Test search quality
!python verify_search_works.py

# Start API
!python run_api.py

# Check GPU usage
!nvidia-smi
```

## 📚 Resources

- **GitHub:** https://github.com/YOUR_USERNAME/Rag_Video_search_Pipeline
- **UCF101 Dataset:** https://www.crcv.ucf.edu/data/UCF101.php
- **CLIP Model:** https://huggingface.co/openai/clip-vit-base-patch32
- **BLIP Model:** https://huggingface.co/Salesforce/blip-image-captioning-base

## 💡 Tips

1. **Use GPU Runtime:** Runtime → Change runtime type → GPU (T4)
2. **Save to Drive:** Colab sessions timeout after 12 hours
3. **Batch Processing:** Process in chunks if running out of memory
4. **Public URL:** Use ngrok for sharing your API
5. **Cost:** Colab is free! Upgrade to Pro for longer sessions

---

**Enjoy your AI-powered video search! 🎉**